In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

from functions import import_imagedata, get_data, label_oh_tf,  Unwrap, ImageProcessor, IDSWDataSetLoader
from fns4wandb import build_optimizer, set_optimizer, hp_sweep, train_model, train_log, log_test_score, set_lossfn, pipeline, test_loop_batch, batch_loop
from loop_fns import res_loop, test_loop,test_loop_res

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from torchvision.models import vgg16
from torchvision.models import resnet101

import cv2

from torch.utils.data import DataLoader
import wandb
from fns4wandb import train_log, build_optimizer
from copy import deepcopy
from tqdm import tqdm

import random
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

import pickle
from fns4wandb import set_lossfn

In [3]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"

In [4]:
model_resnet = resnet101(weights="IMAGENET1K_V1")#.eval

newmodel = torch.nn.Sequential(*(list(model_resnet.children())[:-1]))
newmodel=newmodel.to(device)
print(newmodel)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [5]:
def metrics(label, prediction): #TypeError: Singleton array tensor(3) cannot be considered a valid collection.
    
    label= np.array(label.cpu())

    predictions_np = prediction.cpu().detach().numpy()
    #y_pred' parameter of f1_score must be an array-like or a sparse matrix. Got 7 instead.
    predicted_classes = np.argmax(predictions_np, axis=0)
    #print('metrics Label:   ', label)
    #print('metrics prediction   ', predicted_classes)
    #avg_f1_score = f1_score(label, predictions_np, average='macro')
    acc = accuracy_score(label, predicted_classes)
    
    return acc

In [6]:
def preprocess_im(img_path):
    IP = ImageProcessor(device='cpu')
    img = cv2.imread(img_path) #
    img = IP.blank_padding(img, (224,224))
    img = IP.to_tensor(img).to(device)
    return img


In [7]:
class Squeeze(nn.Module):
    def __init__(self):
        super(Squeeze, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        x = x.squeeze(0)
        x = x.squeeze(1)
        x = x.squeeze(1)
        return x

In [30]:


#run_title = "IDSW_resnet_fine_hp_80_112023"

"""config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        #'dropout':{
        #    'values': [0.5, 0.4, 0.3]
        #},

        'epochs':{
            'value': 70
        },

        'first_lin_lay':{
            'values':[248832]
        },
        'optimizer': {
            'values': ['adam']
        },
            'learning_rate': {
                # a flat distribution between 0 and 0.1
                'distribution': 'log_uniform_values',
                'min': 5e-5,
                'max': 1e-2
            },
        'loss_fn': {
            'values': ['CrossEntropy', 'MSE'] #'MSE', 
        },
        'data_set':{
            'values':['Augmented']
        },
            'scheduler': {
            'values': [0.1, 0.01, 0.001]
        },
        'ks': {
            'values': [(3,5)]
        },

        'channels':{
            'values': [3]
        },
        'num_classes': {
            'values': [11]
        },
        'model_name' : {'values': ['resnet_fine']},
        'channels' : {'values': [3]},
        'image_path': {
            'values': [r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/']
        }
        }
    }"""

config = dict(
    epochs= 120, #30, 
    learning_rate =1e-4,
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 1.0e-03,
    ks = (3,5),
    scheduler=0.5,
    f_lin_lay = 2048,#8192,#5958,#65536,#8192,#7168, #1024*7 = 7168
    dataset = 'IDSW_Aug_22672',
    loss_fn = 'CrossEntropy',
    model = "resnet100"
    
)


     

col_dict = {
    'colour': 'colour',
    'size': [226,72],
    'padding': 5,
    'model_size': 'resnet'
}

title = f"IDSWAug_resnet_e120_{col_dict['size']}_DL_140224"
save_dict = {'Run' : title,
            'Current_Epoch': 0,
            'save_location' : r'pickles/'}


In [9]:
#!wandb login 2bf372d1273e4af99733e10529509d9b252efe88

In [10]:
wandb.login()

wandb: Currently logged in as: naughticalnonsence (antvis). Use `wandb login --relogin` to force relogin


True

In [11]:
#sweep_id = wandb.sweep(config, project=f"{run_title}")

In [12]:
def batch_s1(data):
    for i in data:
        i = i.unsqueeze(i, 0)
    return data

In [13]:
from architectures import PrintLayer, Flattern
def train(device,col_dict, save_dict, config=None):
    # lists for save dict
    t_loss_list = []
    v_loss_list =[]
    t_predict_list = []
    t_label_list = []
    v_predict_list = []
    v_label_list = []
    t_accuracy_list= []
    v_accuracy_list= []
    
    with wandb.init(config=config):
        config = wandb.config

        x_train, y_train, x_val, y_val, x_test, y_test = get_data(file_path= r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/', seed= random.randint(0, 50))
        
        
        #x_train_loader = IDSWDataSetLoader(x_train, y_train, col_dict=col_dict, device=device) #x_train, y_train, col_dict=col_dict, device=device
        #train_loader = DataLoader(x_train_loader, shuffle=True, batch_size=4)
        #x_test_loader = IDSWDataSetLoader(x_test, y_test, col_dict=col_dict, device=device)
        #train_loader = DataLoader(x_test_loader, shuffle=True, batch_size=4)
        #x_val_loader = IDSWDataSetLoader(x_val, y_val, col_dict=col_dict, device=device)
        #train_loader = DataLoader(x_val_loader, shuffle=True, batch_size=4)
        
        #model =smallnet3(in_chan=3, f_lin_lay=67968, l_lin_lay=11, ks=(3,5)).to(device) #10368
        resnet = nn.Sequential(
            newmodel,
            Flattern(),
            Flattern(),
            nn.Linear(config['f_lin_lay'],11), #2048
            nn.Softmax(dim=0),
        )
    
        resnet.to(device)
        model=resnet
        
        if config.loss_fn == 'MSE':
            loss_fn = nn.MSELoss()
        elif config.loss_fn == 'CrossEntropy':
            loss_fn = nn.CrossEntropyLoss()

        e_count = 0
         # *

        optimizer = build_optimizer(model, config.optimizer, config.learning_rate, config.weight_decay)

        for epoch in tqdm(range(config.epochs)):
            # current_loss, predict_list, num_correct, label_list, model, optimizer
            #t_loss, t_predict_list_, t_num_correct, t_label_list_, model, optimizer = batch_loop(model, train_loader, epoch, loss_fn, device, col_dict, num_classes=11, optimizer=optimizer)
            t_loss, t_predict_list_, t_num_correct, t_label_list_, model, optimizer = res_loop(model, x_train, y_train, epoch, loss_fn, device, col_dict, num_classes=11, optimizer=optimizer)

            t_accuracy = (t_num_correct /len(x_train))*100
            t_loss_list.append(t_loss)
            t_predict_list.append(t_predict_list_)
            t_label_list.append(t_label_list_)
            t_accuracy_list.append(t_accuracy)

            #v_loss, v_predict_list_, v_num_correct, v_label_list_= batch_loop(model, x_val_loader, epoch, loss_fn, device,col_dict,num_classes=11, train=False)
            v_loss, v_predict_list_, v_num_correct, v_label_list_= res_loop(model, x_val, y_val, epoch, loss_fn, device,col_dict,num_classes=11, train=False)

            v_accuracy= (v_num_correct / len(x_val))*100
            v_loss_list.append(v_loss)
            v_predict_list.append(v_predict_list_)
            v_label_list.append(v_label_list_)
            v_accuracy_list.append(v_accuracy)

            t_avg_loss =t_loss/len(x_train)
            v_avg_loss = v_loss /len(x_val)

            e_count +=1
            # logging
            wandb.log({'avg_train_loss': t_avg_loss, 'epoch':epoch})
            wandb.log({'avg_val_loss': v_avg_loss, 'epoch':epoch})

            wandb.log({'train_loss': t_loss, 'epoch':epoch})
            wandb.log({'val_loss': v_loss, 'epoch':epoch})

            wandb.log({'train_correct': t_num_correct, 'epoch':epoch})
            wandb.log({'val_correct': v_num_correct, 'epoch':epoch})

            wandb.log({'train_accuracy_%': t_accuracy, 'epoch':epoch})
            wandb.log({'val_accuracy_%': v_accuracy, 'epoch':epoch})

            wandb.log({'t_labels': t_label_list, 'epoch':epoch})
            wandb.log({'v_labels': v_label_list, 'epoch':epoch})

            wandb.log({'t_predictions': t_predict_list, 'epoch':epoch})
            wandb.log({'v_predictions': v_predict_list, 'epoch':epoch})

            # add lists to save dict after all epochs run
    save_dict['Current_Epoch'] = config['epochs']
    save_dict['training_samples'] = len(x_train)# should this be the whole list for future graphs...?
    save_dict['validation_samples'] = len(x_val)
    save_dict['t_loss_list'] = t_loss_list #[c.to('cpu') for c in t_loss_list]
    save_dict['t_predict_list'] = [[c.to('cpu') for c in k]for k in t_predict_list] #[[c.to('cpu') for c in k]for k in t_predict_list]  # [c.to('cpu') for c in t_predict_list] 
    save_dict['t_accuracy_list'] = t_accuracy_list #
    save_dict['v_loss_list'] = v_loss_list #[c.to('cpu') for c in v_loss_list]
    save_dict['v_predict_list'] = [[c.to('cpu') for c in k]for k in v_predict_list]#[[c.to('cpu') for c in k]for k in v_predict_list] # [c.to('cpu') for c in v_predict_list]
    save_dict['v_accuracy_list'] = v_accuracy_list #
    save_dict['t_labels'] = t_label_list #[[c.to('cpu') for c in k]for k in t_label_list]
    save_dict['v_labels'] = v_label_list #[[c.to('cpu') for c in k] for k in v_label_list]
    
    title = save_dict['Run']
    #test_predictions, test_y, test_accuracy = test_loop_batch(model, x_test_loader, loss_fn, device, col_dict, title, config.num_classes)
    test_predictions, test_y, test_accuracy = test_loop_res(model, x_test, y_test, loss_fn, device, col_dict, title, num_classes=11)

    save_dict['test_predictions']= [c.to('cpu') for c in test_predictions]
    save_dict['test_labels'] = test_y
    save_dict['test_acc'] = test_accuracy

    
    with open(f"/its/home/nn268/antvis/antvis/optics/pickles/{title}.pkl", 'wb+') as f:
        pickle.dump(save_dict, f)
        
    return model

In [14]:
# Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)

"""W= 128 #512 # input vol
K = 5 #1 # kernal size
P =0 # padding
S = 1# stride
convOut =(W-K+(2*P))/S+1
print(convOut)"""

'W= 128 #512 # input vol\nK = 5 #1 # kernal size\nP =0 # padding\nS = 1# stride\nconvOut =(W-K+(2*P))/S+1\nprint(convOut)'

In [ ]:

#model, save_dict = pipeline(config['loss_fn'])
model = train(device,col_dict, save_dict, config=config)
"""def tr(config=None):
    with wandb.init(config=config):
        config = wandb.config
        model, save_dict = pipeline(config)
        

wandb.agent(sweep_id, tr, count=20)"""

 75%|▊| 90/120 [1:06:31

In [16]:
#  (1x11648 and 46592x100)
a = 16384*4
print(a/11)

5957.818181818182


In [17]:
"""#model,save_dict = pipeline(config) #7,168

def tr(config=None):
    print('1')
    with wandb.init(config=config):
        print('2')
        config = wandb.config
        print('3')
        model, save_dict = pipeline(config)
        
sweep_id = wandb.sweep(config, project=f"{run_title}"),
#wandb.agent(sweep_id, tr, count=20)"""

'#model,save_dict = pipeline(config) #7,168\n\ndef tr(config=None):\n    print(\'1\')\n    with wandb.init(config=config):\n        print(\'2\')\n        config = wandb.config\n        print(\'3\')\n        model, save_dict = pipeline(config)\n        \nsweep_id = wandb.sweep(config, project=f"{run_title}"),\n#wandb.agent(sweep_id, tr, count=20)'

In [18]:
!nvidia-smi

Tue Feb 13 17:43:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:17:00.0 Off |                  Off |
|  0%   47C    P8              27W / 450W |   5214MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [19]:
# confusion matrix


def plot_confusion(predictions:list, actual:list, title:str):
    predict_list = [int(t.argmax()) for t in predictions]
    actual = [int(l.argmax()) for l in actual]

    actual = np.array(actual)
    predict_list = np.array(predict_list)


    #FixedLocator locations (3), usually from a call to set_ticks, does not match the number of labels (11).
    print(f'\n     {title}')
    train_epoch_matrix = confusion_matrix(actual, predict_list, labels= [0,1,2,3,4,5,6,7,8,9,10])
    disp= ConfusionMatrixDisplay(train_epoch_matrix, display_labels=[0,1,2,3,4,5,6,7,8,9,10])
    disp.plot()
    plt.show()


In [20]:
t_predict = save_dict['t_predict_list']
t_labels = save_dict['t_labels']

v_predict = save_dict['v_predict_list'] # WHY IS THERE NOTHING IN V OREDICT LIST!
v_labels = save_dict['v_labels']

plot_confusion(t_predict, t_labels, 'Train Confusion Matrix')
plot_confusion(v_predict, v_labels, 'Validation Confusion Matrix')

AttributeError: 'list' object has no attribute 'argmax'

In [ ]:
class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        print(x)
        return x

In [ ]:
"""class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        print(x.shape)
        return x
    
    
def pipeline(config): 
    
    resnet = nn.Sequential(
            newmodel,
            Squeeze(),
            nn.Linear(2048,11), #2048
            nn.Softmax(dim=0),
        )
    
    resnet.to(device)
    model=resnet
    loss_list=[]
    model, save_dict = train_model(model,x_train, x_val, y_train, y_val, config) #train_dl, val_dl""

    return model, save_dict"""

"""with wandb.init(project=title, config=config):
    config = wandb.config
    model = resnet

    model, save_dict = train_model(model,x_train, x_val, y_train, y_val, config) #train_dl, val_dl""

return model, save_dict"""

In [ ]:

"""
def train_model(model,  x_train, x_val, y_train, y_val, config, best_acc=0): #train_dl, val_dl, 
    #wandb.watch(model, log='all', log_freq=10)
    #print(config)
    loss_fn = set_lossfn(config['loss_fn']) # ****
    
    lr = config['learning_rate'] #1e-5 #config.learning_rate
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)#build_optimizer(model, optimizer=torch.optim.Adam(model.parameters(), lr=lr))#config.optimizer, config.learning_rate, config.weight_decay)
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=config['scheduler'], last_epoch=-1) #gamma=config.scheduler, last_epoch=-1)
    
    ####
    
    
    model.train()
    
    
    #losses= []
    #predictions = []
    t_loss_list = []
    v_loss_list = []
    t_predict_list = []
    v_predict_list = []
    t_accuracy_list = []
    v_accuracy_list = []
    t_label_list = []
    v_label_list = []
    #labels = []
    
    total_epochs = 0
    for epoch in tqdm(range(config['epochs'])): #config.epochs)):
        print('E   ', epoch)
        t_correct = 0
        v_correct = 0
    
        if epoch == 0:
            model = model.to('cpu')
            best_model = deepcopy(model)
            model = model.to(device)
            
            
        #train_ids = random.shuffle(train_ids)
        #print(type(train_ids))
        print('training...')
        for idx, img in enumerate(x_train): 
            model.train()

            x = preprocess_im(img)

            train_prediction = model.forward(x)
            
            y = label_oh_tf(y_train[idx], device=device, num_classes=11) # use same index val to index y (labels) and turn into onehot encoded label

            """
            #if idx % 1000 == 0:
            #    print(idx, ' / ', len(x_train))
            #    !nvidia-smi
            """
            #print(train_prediction, train_label)
        
        
            t_loss = loss_fn(train_prediction, y)

            if train_prediction.argmax() == y.argmax():
                t_correct+=1

            
            optimizer.zero_grad()
            t_loss.backward()
            
            optimizer.step()
            scheduler.step()
            
            t_loss_list.append(t_loss.to('cpu'))
            t_predict_list.append(train_prediction.to('cpu'))
            t_label_list.append(y.to('cpu'))
            
            train_acc = (t_correct / len(x_train))
        t_accuracy_list.append(train_acc)
            
            # accuracy at every step. every epoch / by x_train
            
        print('validating...')
        
        for idx, img in enumerate(x_val):
            model.eval()
            x = preprocess_im(img)

            val_prediction = model.forward(x)
            
            y = label_oh_tf(y_val[idx], device=device, num_classes=11)
            """
            #if idx % 100 == 0:
            #    print(idx, ' / ', len(x_val))
            #    !nvidia-smi
            """

            v_loss = loss_fn(val_prediction, y)
            
            if val_prediction.argmax() == y.argmax():
                v_correct +=1
                

            v_loss = v_loss.to('cpu')
            v_loss_list.append(v_loss.item())
            
            
            val_prediction = val_prediction.to('cpu')
            v_predict_list.append(val_prediction.detach().numpy())
            
            v_label_list.append(y.to('cpu').detach().numpy())
            
        val_acc = (v_correct / len(y_val))
        v_accuracy_list.append(val_acc)
            
        total_epochs += epoch

        if val_acc > best_acc:
    
            #print('Start Of SAve -----------------------------')
            #!nvidia-smi

            best_acc = val_acc
            
            model = model.to('cpu')
            best_model = model#deepcopy(model)
            model = model.to(device)
            

            
            save_dict['Current_Epoch'] += config['epochs']
            save_dict['training_samples'] = len(x_train)
            save_dict['validation_samples'] = len(x_val)
            save_dict['t_loss_list'] = t_loss_list
            save_dict['t_predict_list'] = t_predict_list  
            save_dict['t_accuracy_list'] = t_accuracy_list  #
            save_dict['v_loss_list'] = v_loss_list
            save_dict['v_predict_list'] = v_predict_list  #
            save_dict['v_accuracy_list'] = v_accuracy_list  #
            save_dict['t_labels'] = t_label_list
            save_dict['v_labels'] = v_label_list
            
            model_architecture = [nn.Sequential(
                            model_vgg16,
                            Squeeze(),
                            nn.Linear(4096,11),
                            nn.Softmax(dim=0),
                        )]

            save_dict['model.state_dict'] = model.state_dict()# .to('cpu')
            #save_dict['model_architecture_untrained'] = model_architecture

            title = save_dict['Run']
            with open(f'{save_location}{title}.pkl', 'wb+') as f:
                pickle.dump(save_dict, f)
            
            print('improvment in metrics. model saved')

            #print('END Of SAve -----------------------------')
            #!nvidia-smi

        

        #if (epoch+1)%2==0:
            #train_log(t_loss, v_loss, epoch)
            #wandb.log({'train_accuracy_%': train_acc, 'epoch':epoch})
            #wandb.log({'val_accuracy_%': val_acc, 'epoch':epoch})
            
    model = best_model
    #labels = zip(t_label_list, v_label_list)
    #losses = zip(t_loss_list, v_loss_list)
    #predictions = zip(t_predict_list, v_predict_list)
    return model,save_dict
"""





In [ ]:
"""model =nn.Sequential(
    PrintLayer(),
    model_vgg16,
    PrintLayer(),
    Squeeze(),
    PrintLayer(),
    nn.Linear(2048,11),
    PrintLayer(),
    nn.Softmax(dim=0),
    PrintLayer()

).to(device)"""

In [ ]:

x = preprocess_im(x_train[0])


train_prediction = model.forward(x)

In [ ]:
model.to('cpu').state_dict()